In [ ]:
%pip install lxml
%pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.read_csv(
    'https://dqydj.com/scripts/cps/2022_income_calculators/cps_2022_households.csv',)

In [ ]:
pd.read_csv(
    'https://dqydj.com/scripts/scf/2020_wealth_calculators/scf_2020_age.csv',)

In [ ]:
# https://dqydj.com/scripts/cps/2022_income_calculators/2022_income_by_age.html
# Inspecting scripts led here
# https://dqydj.com/scripts/cps/2022_income_calculators/2022_income_by_age.html
# Then reading the js gave me the below link:
df = pd.read_csv(
    'https://dqydj.com/scripts/cps/2022_income_calculators/cps_2022_income_by_age.csv',
    
)
df = df.set_index('Pct')
df

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio
pio.renderers.default = 'iframe'

In [ ]:
fig = go.Figure(data=[go.Surface(z=df.values)])
fig.show()

In [ ]:
import statsmodels

In [ ]:
def smooth_matrix_along_rows_lowess(mat, *args, **kwargs):
    smoothed = np.empty_like(mat)
    for i, row in enumerate(mat):
        smoothed[i] = statsmodels.nonparametric.smoothers_lowess.lowess(
            endog=row,
            exog=np.arange(len(row)),
            return_sorted=False,
            *args,
            **kwargs,
        )
    return smoothed

In [ ]:
fig = go.Figure(
    data=[
        go.Surface(
            z=smooth_matrix_along_rows_lowess(df.values),
            colorscale='ice_r',
#             contours = {
#                 #"y": {"show": True, "size": 0.01, "color":"black"},
#                 #"z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.01}
#             }
        )
    ]
)
fig.update_layout(
    title="Income by Income Percentile and Age",
)
fig.update_traces(
    hovertemplate="""Age: %{x}<br>Percentile %{y}<br>Income: %{z}""",
)
fig.show()

In [ ]:
np.median(smooth_matrix_along_rows_lowess(df.values))

In [ ]:
smooth_matrix_along_rows_lowess(df.values).mean()

In [ ]:
fig = go.Figure(data=[go.Surface(z=smooth_matrix_along_rows_lowess(df.values.T, frac=0.1))])
fig.show()

In [ ]:
fig = go.Figure(data =
    go.Contour(
        z=smooth_matrix_along_rows_lowess(df.values),
        colorscale='ice_r',
    ))
fig.show()

In [ ]:
px.scatter(df['24'], trendline="lowess", trendline_options=dict(frac=0.1))

In [ ]:
px.scatter(df.T[0.99], trendline="lowess")

In [ ]:
px.scatter(df.T[0.99], trendline="ewm", trendline_options=dict(halflife=2))

In [ ]:
px.scatter(df.T[0.99], trendline="rolling", 
                 trendline_options=dict(window=10, win_type="gaussian", function_args=dict(std=2)))

In [ ]:
fig = go.Figure(data=[go.Surface(z=np.log(1 + df.values))])
fig.show()